##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# DQN C51/Rainbow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/9_c51_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/9_c51_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/9_c51_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/9_c51_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

Este exemplo mostra como treinar um agente [Categorical DQN (C51)](https://arxiv.org/pdf/1707.06887.pdf) no ambiente Cartpole usando a biblioteca TF-Agents.

![Ambiente Cartpole](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/images/cartpole.png?raw=1)

Como pré-requisito, confira o [tutorial do DQN](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/1_dqn_tutorial.ipynb). Este tutorial presume que você está familiarizado com o tutorial do DQN, focando principalmente nas diferenças entre o DQN e o C51.


## Configuração


Se você ainda não instalou o tf-agents, execute:

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents
!pip install pyglet

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import categorical_q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

## Hiperparâmetros

In [ ]:
env_name = "CartPole-v1" # @param {type:"string"}
num_iterations = 15000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_capacity = 100000  # @param {type:"integer"}

fc_layer_params = (100,)

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
gamma = 0.99
log_interval = 200  # @param {type:"integer"}

num_atoms = 51  # @param {type:"integer"}
min_q_value = -20  # @param {type:"integer"}
max_q_value = 20  # @param {type:"integer"}
n_step_update = 2  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## Ambiente

Carregue o ambiente como antes, sendo um para treinamento e outro para avaliação. Aqui, usamos CartPole-v1 (em vez do CartPole-v0 do tutorial de DQN), que tem uma maior recompensa máxima de 500, e não 200.

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## Agente

O C51 é um algoritmo de Q-learning baseado em DQN. Como o DQN, ele pode ser usado em qualquer ambiente com um espaço de ação discreto.

A principal diferença entre o C51 e o DQN é que, em vez de apenas prever o Q-value para cada par de estado-ação, o C51 prevê um modelo de histograma para a distribuição de probabilidade do Q-value:

![Exemplo de distribuição C51](images/c51_distribution.png)

Ao aprender a distribuição em vez de somente o valor esperado, o algoritmo consegue ficar mais estável durante o treinamento, levando a um melhor desempenho final. Isso se aplica principalmente a situações com distribuições de valores bimodais ou até multimodais, em que uma única média não fornece um quadro preciso.

Para treinar com distribuições de probabilidade em vez de valores, o C51 precisa realizar algumas computações distributivas complexas para calcular a função de perda. Não se preocupe, isso tudo é feito para você no TF-Agents!

Para criar um Agente C51, primeiro precisamos criar uma `CategoricalQNetwork`. A API da `CategoricalQNetwork` é igual a da `QNetwork`, mas com um argumento `num_atoms` adicional. Isso representa o número de pontos de suporte nas nossas estimativas de distribuição de probabilidade. (A imagem acima inclui 10 pontos de suporte, cada um representado por uma barra azul vertical.) Como você pode ver pelo nome, o número padrão de átomos é 51.


In [ ]:
categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    num_atoms=num_atoms,
    fc_layer_params=fc_layer_params)

Também precisamos de um `optimizer` para treinar a rede que acabamos de criar e uma variável `train_step_counter` para acompanhar quantas vezes a rede foi atualizada.

Observe que outra diferença significativa do `DqnAgent` vanilla é que agora precisamos especificar `min_q_value` e `max_q_value` como argumentos. Eles especificam os valores mais extremos do suporte (em outras palavras, o átomo mais extremo dos 51 em qualquer lado). Certifique-se de escolher esses valores de maneira adequada para seu ambiente específico. Aqui, usamos -20 e 20.

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter)
agent.initialize()

Uma última coisa a observar é que também adicionamos um argumento para usar atualizações de n-step com $n$ = 2. No Q-learning de um único passo ($n$ = 1), só computamos o erro entre os Q-values no timestep atual e no próximo usando o retorno de único passo (com base na equação de otimalidade do Bellman). O retorno de um único passo é definido como:

$G_t = R_{t + 1} + \gamma V(s_{t + 1})$

onde definimos $V(s) = \max_a{Q(s, a)}$.

As atualizações de n-step envolvem ampliar a função de retorno de um único passo padrão $n$ vezes:

$G_t^n = R_{t + 1} + \gamma R_{t + 2} + \gamma^2 R_{t + 3} + \dots + \gamma^n V(s_{t + n})$

As atualizações de n-step permitem que o agente se inicialize mais no futuro e, com o valor certo de $n$, isso geralmente leva a um aprendizado mais rápido.

Embora as atualizações de n-step e C51 sejam combinadas frequentemente com o replay priorizado para formar o núcleo do [agente Rainbow](https://arxiv.org/pdf/1710.02298.pdf), não vimos nenhuma melhoria mensurável ao implementar o replay priorizado. Além disso, descobrimos que, ao combinar nosso agente C51 com apenas as atualizações de n-step, nosso agente tem o mesmo desempenho que outros agentes Rainbow na amostra dos ambientes Atari que testamos.

## Métricas e avaliação

A métrica mais comum usada para avaliar uma política é o retorno médio. O retorno é a soma das recompensas obtidas ao executar uma política em um ambiente para um episódio, e geralmente calculamos isso em alguns episódios. Podemos computar a métrica de retorno médio da seguinte maneira:


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

compute_avg_return(eval_env, random_policy, num_eval_episodes)

# Please also see the metrics module for standard implementations of different
# metrics.

## Coleta de dados

Como no tutorial DQN, configuramos o buffer de replay e a coleta de dados inicial com a política aleatória.

In [ ]:
#@test {"skip": true}
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

def collect_step(environment, policy):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)

  # Add trajectory to the replay buffer
  replay_buffer.add_batch(traj)

for _ in range(initial_collect_steps):
  collect_step(train_env, random_policy)

# This loop is so common in RL, that we provide standard implementations of
# these. For more details see the drivers module.

# Dataset generates trajectories with shape [BxTx...] where
# T = n_step_update + 1.
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)

iterator = iter(dataset)

## Treinamento do agente

O loop de treinamento envolve tanto coletar os dados do ambiente quanto otimizar as redes do agente. Ao longo do caminho, vamos avaliar ocasionalmente a política do agente para ver como está o desempenho.

A execução do código a seguir levará cerca de 7 minutos.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience)

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1:.2f}'.format(step, avg_return))
    returns.append(avg_return)

## Visualização


### Plots

Podemos plotar o retorno em comparação com os passos dos globais para ver o desempenho do nosso agente. Em `Cartpole-v1`, o ambiente dá uma recompensa de +1 a cada timestep em que o pêndulo permanece em pé e, como o número máximo de passos é 500, o retorno máximo possível também é 500.

In [ ]:
#@test {"skip": true}

steps = range(0, num_iterations + 1, eval_interval)
plt.plot(steps, returns)
plt.ylabel('Average Return')
plt.xlabel('Step')
plt.ylim(top=550)

### Vídeos

É útil visualizar o desempenho de um agente ao renderizar o ambiente a cada passo. Antes de fazer isso, vamos criar uma função para incorporar vídeos neste colab.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

O código a seguir visualiza a política do agente por alguns episódios:

In [ ]:
num_episodes = 3
video_filename = 'imageio.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_py_env.render())
    while not time_step.is_last():
      action_step = agent.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_py_env.render())

embed_mp4(video_filename)

O C51 tende a ter um desempenho um pouco melhor do que o DQN no CartPole-v1, mas a diferença entre os dois agentes se torna mais e mais significativa em ambientes cada vez mais complexos. Por exemplo, no benchmark Atari 2600 completo, o C51 demonstra uma melhoria na pontuação média de 126% em comparação com o DQN após a normalização em relação a um agente aleatório. Melhorias adicionais podem ser obtidas ao incluir atualizações de n-step.

Para saber mais sobre o algoritmo C51, confira [A Distributional Perspective on Reinforcement Learning (2017)](https://arxiv.org/pdf/1707.06887.pdf) (Uma perspectiva distributiva do aprendizado por reforço).